For installation of **DSSEX** please follow [installation instructions](./readme.ipynb#Install-DSSEX) or run the next cell.

In [ ]:
# in case you want to install dssex in the environment of Jupyter e.g. Google Colab
%pip install -q dssex

In [ ]:
import dssex
from egrid import create_objects, make_data_frames, check_frames, model_from_frames
import pandas as pd
pd.set_option('display.max_colwidth', None)

def print_frames(frames):
    """Prints pandas.DataFrame instances stored in a dict"""
    messages = frames.get('Message')
    if messages is not None:
        is_error = 1 < messages.level
        errors = messages.loc[is_error].message
        if not errors.empty:
            print('Errors\n')
            for _, error in errors.items():
                print()
                print(error)
            print()
    for key,df in frames.items():
        print()
        print(key)
        display(df)
        
def print_results(res):
    """Prints values returned from function dssex.estimate"""
    messages, vals = res
    if not messages.empty:
        print('\n\n')
        print('Messages')
        display(messages)
    last_index = len(vals) - 2
    for idx, df in enumerate(vals, -1):
        print('\n\n')
        print(f'{"RESULT >> " if idx==last_index else ""}Optimization Step: {idx}')
        for name in ['nodes', 'branches', 'injections']:
            print()
            print(name.title())
            display(df[name])    

# Volt-Var-Control

In [ ]:
schema00 = """
slack +-----------transformer-----------+ n +-----------> heating
      Tlink=tap    y_lo=1e3-1e3j          |                P10=200
                   y_tr=1e-6+1e-6j        |
                                          | 
                                          | 
                         cap ||---------+ n +-------((~)) motor
                          Q10=-10                          P10=160
                          Exp_v_q=2                        Q10=10

#.
Deft(id=tap type=var min=-16 max=16 value=0 is_discrete=True cost=.03)
Defk(id=kcap type=var min=0 max=5 value=0 is_discrete=True cost=.05)
Klink(id_of_injection=cap id_of_factor=kcap part=q)
Vlimit(min=.95)
"""
frames00 = make_data_frames(create_objects(schema00))
print_frames(frames00)

In [ ]:
pd.DataFrame.from_records(check_frames(frames00), columns=['message','level'])

[step_params](./help_params.ipynb)

In [ ]:
model00 = model_from_frames(frames00)
step_params=[
    # first step: minimize voltage violations
    dict(objectives='U', constraints='U'),
    # second step: minimize cost
    dict(objectives='CL', constraints='U', floss=2)
]

results00 = dssex.estimate(model00, step_params)

Results for two steps. 

- Initial power flow calculation
- Optimization

In [ ]:
print_results(results00)